In [1]:
#importing all the required packages
import optuna
from optuna.samplers import GPSampler


import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
import os
from pathlib import Path

import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/')
from model.GRU_model import *
from pipeline_config import *
from utils import *

/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs : {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.0758230781941514
Epochs : 100


In [3]:
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [4]:
def objective(trial):
    with open(train_dataset_path, 'rb') as f:
        train_dataset_dict = pickle.load(f)
        
    with open(test_dataset_path, 'rb') as f:
        test_dataset_dict = pickle.load(f)

    with open(category_id_to_category_name_path, 'rb') as f:
        category_id_to_name_dict = pickle.load(f)
        
    with open(categories_per_indent_path, 'rb') as f:
        categories_per_indent_dict = pickle.load(f)

    with open(son_parent_path, 'rb') as f:
        son_parent_dict = pickle.load(f)
    
    weights_path = weightspath
    #--------------------------------------------------------------------------------------------------------------------------------------#
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights/'

    is_empty = not any(Path(desired_path).iterdir())
    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)

    #--------------------------------------------------------------------------------------------------------------------------------------#

    loss_coef = trial.suggest_float('loss_coef',  1e-10, 1e-1, log=True)
    Lr = trial.suggest_float('Lr', 1e-6, 1e-1, log=True)
    seed = trial.suggest_int('seed', 0 , 100)
    hgru_models = {}

    num_categories = 0
    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                parent_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=seed)
                parent_optimizer = torch.optim.AdamW(Model.parameters(), lr=Lr)
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model=GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=seed)
            optimizer = torch.optim.AdamW(Model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            hgru_models[category_name] = training_and_evaluation(trial, model, train_dataloader, test_dataloader, optimizer, category, parent_weights, loss_coef, path= saving_param_path)
                

    average_error = sum(list(hgru_models.values()))/len(list(hgru_models.values()))
    
    return average_error


In [5]:
study = optuna.create_study(direction="minimize", sampler=GPSampler())
study.optimize(objective, n_trials=50)

/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_83654/2604621552.py:1: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
  study = optuna.create_study(direction="minimize", sampler=GPSampler())
[I 2024-07-08 19:43:29,494] A new study created in memory with name: no-name-7ad32583-dc07-40b0-ac8f-a654ea80647b


Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/hgru/models_weights/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/hgru/models_weights/
num categories: 1
category id|name: 8106.0|All items
num categories: 2
category id|name: 4604.0|All items less energy


/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python

num categories: 3
category id|name: 3531.0|All items less food
num categories: 4
category id|name: 5306.0|All items less food and energy
num categories: 5
category id|name: 4114.0|All items less food and shelter
num categories: 6
category id|name: 5789.0|All items less food, shelter, and energy
num categories: 7
category id|name: 2208.0|All items less food, shelter, energy, and used cars and trucks
num categories: 8
category id|name: 2047.0|All items less medical care
num categories: 9
category id|name: 7199.0|All items less shelter
num categories: 10
category id|name: 3242.0|Apparel less footwear
num categories: 11
category id|name: 704.0|Commodities
num categories: 12
category id|name: 2665.0|Durables
num categories: 13
category id|name: 1868.0|Education and communication
num categories: 14
category id|name: 7558.0|Energy
num categories: 15
category id|name: 4934.0|Food
num categories: 16
category id|name: 9257.0|Food and beverages
num categories: 17
category id|name: 5463.0|Fuels an

[I 2024-07-08 20:06:07,546] Trial 0 finished with value: 0.10197747781739704 and parameters: {'loss_coef': 7.327069662479958e-06, 'Lr': 0.008910738395418387, 'seed': 74}. Best is trial 0 with value: 0.10197747781739704.


...

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/hgru/models_weights/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/hgru/models_weights/
num categories: 1
category id|name: 8106.0|All items
num categories: 2
category id|name: 4604.0|All items less energy


/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
/Users/mvilenko/Library/Python/3.9/lib/python

num categories: 3
category id|name: 3531.0|All items less food
num categories: 4
category id|name: 5306.0|All items less food and energy
num categories: 5
category id|name: 4114.0|All items less food and shelter
num categories: 6
category id|name: 5789.0|All items less food, shelter, and energy
num categories: 7
category id|name: 2208.0|All items less food, shelter, energy, and used cars and trucks
num categories: 8
category id|name: 2047.0|All items less medical care
num categories: 9
category id|name: 7199.0|All items less shelter
num categories: 10
category id|name: 3242.0|Apparel less footwear
num categories: 11
category id|name: 704.0|Commodities
num categories: 12
category id|name: 2665.0|Durables
num categories: 13
category id|name: 1868.0|Education and communication
num categories: 14
category id|name: 7558.0|Energy
num categories: 15
category id|name: 4934.0|Food
num categories: 16
category id|name: 9257.0|Food and beverages
num categories: 17
category id|name: 5463.0|Fuels an

[W 2024-07-09 11:05:52,894] Trial 42 failed with parameters: {'loss_coef': 3.3897015859833866e-09, 'Lr': 0.1, 'seed': 0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/mvilenko/Library/Python/3.9/lib/python/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_83654/1347830824.py", line 75, in objective
    hgru_models[category_name] = training_and_evaluation(trial, model, train_dataloader, test_dataloader, optimizer, category, parent_weights, loss_coef, path= saving_param_path)
  File "/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/hgru/utils.py", line 131, in training_and_evaluation
    epoch_train_loss, epoch_train_mse_loss = training_loop(model, train_dataloader, optim, parent_weights, loss_coef)
  File "/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/US/hgru/utils.py", line 84, i

KeyboardInterrupt: 

In [ ]:
# Get the best trial
best_trial = study.best_trial

# Print the best trial
print('Best trial:')
print(f'Value: {best_trial.value}')
print('Parameters:')
for key, value in best_trial.params.items():
    print(f'{key}: {value}')